In [29]:
import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr
import datetime
from bs4 import BeautifulSoup
from pykrx import stock
import math

In [30]:
api_key = '133579488f1e7f9a44707b2a7d2be9999840935a'
dart = OpenDartReader(api_key)

In [31]:
day_buy = '2021-04-05'

In [32]:
list_d002 = dart.list(end=day_buy, kind='D', kind_detail='D002')
list_d002.head()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00989619,알테오젠,196170,K,임원ㆍ주요주주특정증권등소유상황보고서,20210405003144,박순재,20210405,
1,00989619,알테오젠,196170,K,임원ㆍ주요주주특정증권등소유상황보고서,20210405003129,정혜신,20210405,
2,01199550,현대에너지솔루션,322000,Y,주식등의대량보유상황보고서(약식),20210405003120,국민연금공단,20210405,
3,01205842,현대건설기계,267270,Y,주식등의대량보유상황보고서(약식),20210405003112,국민연금공단,20210405,
4,00166227,화승인더스트리,006060,Y,주식등의대량보유상황보고서(약식),20210405003107,국민연금공단,20210405,


In [33]:
list_report_nm = list_d002[(list_d002['report_nm'] == '임원ㆍ주요주주특정증권등소유상황보고서') & ((list_d002['corp_cls'] == 'Y') | (list_d002['corp_cls'] == 'K' ))]
list_report_nm.head()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00989619,알테오젠,196170,K,임원ㆍ주요주주특정증권등소유상황보고서,20210405003144,박순재,20210405,
1,00989619,알테오젠,196170,K,임원ㆍ주요주주특정증권등소유상황보고서,20210405003129,정혜신,20210405,
12,00989619,알테오젠,196170,K,임원ㆍ주요주주특정증권등소유상황보고서,20210405003075,이재상,20210405,
21,00606664,케이엔더블유,105330,K,임원ㆍ주요주주특정증권등소유상황보고서,20210405003021,오범석,20210405,
24,01335851,박셀바이오,323990,K,임원ㆍ주요주주특정증권등소유상황보고서,20210405003009,정광준,20210405,


In [34]:
result = pd.DataFrame(columns=['corp_name', 'corp_code', 'mdf_stk_cnt'])
for idx, row in list_report_nm.iterrows():
    html = dart.document(row['rcept_no'])
    soup = BeautifulSoup(html, 'html5lib')
    sec3_2 = soup.select('section-3')[2]
    rpt_rsn = sec3_2.findAll('tu', {'aunit': 'RPT_RSN'})[-1]
    mdf_stk_cnt = sec3_2.findAll('te', {'acode': 'MDF_STK_CNT'})[-1]
    if (rpt_rsn.contents[0] == '장내매수(+)'):
        tmp = pd.DataFrame(data=[[row['corp_name'], row['corp_code'], row['stock_code'], mdf_stk_cnt.contents[0]]], columns=['corp_name', 'corp_code', 'stock_code', 'mdf_stk_cnt'])
        result = result.append(tmp, ignore_index=True)
result

,corp_name,corp_code,mdf_stk_cnt,stock_code
0,한국큐빅,00160010,"3,320",021650
1,유유제약,00144818,"1,000",000220
2,한화,00160588,700,000880
3,베셀,00813389,"1,000",177350
4,티웨이항공,00671376,"1,000",091810
5,세아제강,01325979,440,306200
6,대교,00108913,600,019680


In [35]:
def getFinstate(corp, year, quarter):
    # "11013"=1분기보고서,  "11012"=반기보고서,  "11014"=3분기보고서, "11011"=사업보고서
    reprt_code = {1: '11013',
                  2: '11012',
                  3: '11014',
                  4: '11011'}
    return dart.finstate(corp, year, reprt_code=reprt_code)

In [36]:
def getProfitFromFinstate(finstate):
    if finstate is None:
        return 0
    finstate = finstate.set_index('account_nm')
    profit = finstate.loc['당기순이익', 'thstrm_amount']
    if not isinstance(profit, str):
        profit = profit[0]
    profit = int(profit.replace(',',''))
    return profit

In [37]:
def getAssetFromFinstate(finstate):
    if finstate is None:
        return 0
    finstate = finstate.set_index('account_nm')
    asset = finstate.loc['자본총계', 'thstrm_amount']
    if not isinstance(asset, str):
        asset = asset[0]
    asset = int(asset.replace(',',''))
    return asset

In [38]:
result2 = pd.DataFrame()
for idx, row in result.iterrows():
    for year in [2018, 2019, 2020]:
        finstate = dart.finstate(row['corp_code'], year, reprt_code='11011')
        col_name = '당기순이익(' + str(year) + ')'
        row[col_name] = getProfitFromFinstate(finstate)
    
    finstate = dart.finstate(row['corp_code'], 2020, reprt_code='11011')
    row['자본총계'] = getAssetFromFinstate(finstate)
    result2 = result2.append(row)
result2

,corp_code,corp_name,mdf_stk_cnt,stock_code,당기순이익(2018),당기순이익(2019),당기순이익(2020),자본총계
0,00160010,한국큐빅,"3,320",021650,8.121472e+08,3.347893e+09,1.050806e+10,6.730033e+10
1,00144818,유유제약,"1,000",000220,4.312110e+09,5.563343e+09,2.412605e+09,1.170364e+11
2,00160588,한화,700,000880,7.993400e+11,2.310540e+11,7.405010e+11,1.850720e+13
3,00813389,베셀,"1,000",177350,1.281373e+09,-1.104672e+10,-2.266139e+09,1.892078e+10
4,00671376,티웨이항공,"1,000",091810,3.788034e+10,-4.320999e+10,-1.378705e+11,1.169858e+11
5,01325979,세아제강,440,306200,7.030156e+09,2.485199e+10,3.305946e+10,5.952522e+11
6,00108913,대교,600,019680,1.922452e+10,1.715526e+10,-1.818160e+10,5.588865e+11


In [39]:
market_cap = stock.get_market_cap_by_ticker(day_buy.replace('-',''))
print(market_cap.head())
# market_cap.loc['000660', '시가총액']

            종가             시가총액       거래량           거래대금       상장주식수
티커                                                                  
005930   85400  509819429770000  16255990  1385160353200  5969782550
000660  143000  104104338195000   2616440   373549552500   728002365
005935   75500   62127945850000   1486188   111925136300   822886700
035420  377000   61927299915000    421508   158651864000   164263395
051910  817000   57673944231000    188784   155235548000    70592343


In [40]:
result3 = pd.DataFrame()
for idx, row in result2.iterrows():
    row['순이익가치'] = str(math.floor(
         (row['당기순이익(2020)'] * 3 + row['당기순이익(2019)'] * 2 + row['당기순이익(2018)']) / 6 * 10
         ))
    row['기업가치'] = str(math.floor(int(row['순이익가치']) * 0.6 +  int(row['자본총계']) * 0.4))
    row['시가총액'] = str(market_cap.loc[row['stock_code'], '시가총액'])
    row['가치/시총'] = '{:.2f}%'.format(int(row['기업가치']) / int(row['시가총액']) * 100)
    result3 = result3.append(row)
result3

,corp_code,corp_name,mdf_stk_cnt,stock_code,가치/시총,기업가치,당기순이익(2018),당기순이익(2019),당기순이익(2020),순이익가치,시가총액,자본총계
0,00160010,한국큐빅,"3,320",021650,75.54%,65952232349,8.121472e+08,3.347893e+09,1.050806e+10,65053503671,87312006420,6.730033e+10
1,00144818,유유제약,"1,000",000220,104.96%,69491188102,4.312110e+09,5.563343e+09,2.412605e+09,37794352528,66210435280,1.170364e+11
2,00160588,한화,700,000880,469.98%,10885830199999,7.993400e+11,2.310540e+11,7.405010e+11,5804918333333,2316224911500,1.850720e+13
3,00813389,베셀,"1,000",177350,-29.31%,-20042180769,1.281373e+09,-1.104672e+10,-2.266139e+09,-46017485359,68385368480,1.892078e+10
4,00671376,티웨이항공,"1,000",091810,-137.40%,-415356970092,3.788034e+10,-4.320999e+10,-1.378705e+11,-770252131562,302304964460,1.169858e+11
5,01325979,세아제강,440,306200,146.85%,394013401146,7.030156e+09,2.485199e+10,3.305946e+10,259854174896,268313980000,5.952522e+11
6,00108913,대교,600,019680,58.65%,222544844515,1.922452e+10,1.715526e+10,-1.818160e+10,-1682945219,379468768000,5.588865e+11
